In [19]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score , classification_report
from sklearn.compose import make_column_selector, make_column_transformer
 

In [21]:
hr_df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Cases\human-resources-analytics\HR_comma_sep.csv')
hr_df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [22]:
X = hr_df.drop('salary', axis=1)
y = hr_df['salary']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [ ]:
ohe = OneHotEncoder(
    sparse_output=False, 
    drop='first', 
    handle_unknown='ignore'
).set_output(transform='pandas')

ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_exclude=object))
)

nb = GaussianNB()

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

pipe = Pipeline([
    ('CT', ct), 
    ('NB', nb)
])

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)


gcv = GridSearchCV(
    estimator=pipe, 
    cv=kfold,
    scoring='roc_auc',
    param_grid={ 
        'NB__var_smoothing': np.linspace(1e-15, 1, 10),
    }, 
)

gcv.fit(X_train, y_train_encoded)
print('Best ROC AUC Score on training set:', gcv.best_score_)

y_pred = gcv.best_estimator_.predict(X_test)
y_pred_proba = gcv.best_estimator_.predict_proba(X_test)


Best ROC AUC Score on training set: nan
              precision    recall  f1-score   support

        high       0.09      0.95      0.17       401
         low       0.54      0.05      0.09      2155
      medium       0.48      0.04      0.07      1943

    accuracy                           0.12      4499
   macro avg       0.37      0.34      0.11      4499
weighted avg       0.48      0.12      0.09      4499

Test ROC AUC Score: 0.5234794295809511


In [27]:
print("classification_report:\n", classification_report(y_test_encoded, y_pred, target_names=le.classes_))

print("Test ROC AUC Score:\n", roc_auc_score(y_test_encoded, y_pred_proba, multi_class='ovr'))

classification_report:
               precision    recall  f1-score   support

        high       0.09      0.95      0.17       401
         low       0.54      0.05      0.09      2155
      medium       0.48      0.04      0.07      1943

    accuracy                           0.12      4499
   macro avg       0.37      0.34      0.11      4499
weighted avg       0.48      0.12      0.09      4499

Test ROC AUC Score:
 0.5234794295809511
